#Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier,
    ExtraTreesClassifier,
    AdaBoostClassifier,
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
)
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm
import matplotlib.pyplot as plt

train = pd.read_parquet('processed_train.parquet')
test = pd.read_parquet('test.parquet')


1. traintestsplit для валидации
+ еще надо на фолды бить
2. подбор гиперпараметров
3. Надо ли тест менять под трейн?
4. модельки: КНН, лес, логрег = вероятности

In [ ]:
train.describe()

,"('average_values',)","('moda_values',)","('median_values',)","('variance_values',)","('std_values',)","('average_days',)","('moda_days',)","('median_days',)","('variance_days',)","('std_days',)","('label',)"
count,110492.000000,110492.000000,110492.000000,110492.000000,110492.000000,110492.000000,110492.000000,110492.000000,110492.000000,110492.000000,110492.000000
mean,0.776990,0.455403,0.515910,0.469685,0.536357,0.293241,0.103652,0.292771,0.394177,0.478209,0.500000
std,0.090396,0.091140,0.101452,0.191232,0.185307,0.209105,0.196546,0.209145,0.255294,0.251190,0.500002
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.725815,0.394523,0.446620,0.319058,0.393280,0.119588,0.000000,0.118929,0.182574,0.271784,0.000000
50%,0.788846,0.446247,0.514579,0.458197,0.535568,0.260904,0.000000,0.260714,0.329917,0.441036,0.500000
75%,0.832545,0.514199,0.582538,0.602290,0.670963,0.413066,0.115044,0.412143,0.560359,0.661315,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
cols = train.columns

In [ ]:
train.rename(columns = {cols[0]:"average_values", cols[1]:"moda_values", cols[2]:"median_values", cols[3]:"variance_values", cols[4]:"std_values",
                        cols[5]: "average_days", cols[6]: "moda_days", cols[7]: "median_days", cols[8]: "variance_days", cols[9]: "std_days",
                        cols[10]: "label"}, inplace = True)

#Train/validation split

In [ ]:
X = train.drop(columns = ['label'])
y = train['label']

X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size = 0.3,
                                                  stratify = y,
                                                  random_state = 42)

print('Train: ', X_train.shape)
print('Valudation: ', X_val.shape)

Train:  (77344, 10)
Valudation:  (33148, 10)


#ExtraTrees
hyperparametes was chosen by gridSearch, model - by comparing a couple of classification models metrics

In [ ]:
model_rfc =  ExtraTreesClassifier(criterion = 'gini', n_estimators = 150, bootstrap = True)
model_rfc.fit(X_train, y_train) # обучение
y_pred = model_rfc.predict(X_val) # предсказание

print ("AUC-ROC (test) = ", metrics.roc_auc_score(y_val, y_pred))

AUC-ROC (test) =  0.8826475202123808


In [ ]:
probs = model_rfc.predict_proba(X_val)
probs

array([[0.  , 1.  ],
       [0.  , 1.  ],
       [0.94, 0.06],
       ...,
       [0.  , 1.  ],
       [0.06, 0.94],
       [0.87, 0.13]])

# Loading model

In [ ]:
from joblib import Parallel, delayed
import joblib


# Save the model as a pickle in a file
joblib.dump(model_rfc, 'model.pkl')

['model.pkl']